In [1]:
import cv2
import pandas as pd
import numpy as np


# set random seed for reproducibility
np.random.seed(42)


# screen resolution
monitor_width = 3440
monitor_height = 1440

In [2]:
def rescale_frame(frame, scale):

    width = int(frame.shape[1] * scale)
    height = int(frame.shape[0] * scale)

    return cv2.resize(frame, (width, height), interpolation=cv2.INTER_AREA)


def find_scale_factor(frame, monitor_width, monitor_height):
    """
    Rescales the frame if it is larger than the monitor 
    """

    frame_width = frame.shape[1]
    frame_height = frame.shape[0]

    if (frame_width > monitor_width) or (frame_height > monitor_height):
        with_scale = monitor_width / frame_width
        height_scale = monitor_height / frame_height
        return min(with_scale, height_scale)

    return 1
    

In [3]:
df_frames = pd.read_csv('../data/tmp_tracking.csv')

### Pedal position while horizontal pedals

In [4]:
frames_parallel_pedals = df_frames[df_frames['parallel_pedals'] == True]

# sample a frame from frames_parallel_pedals
sample_frame = frames_parallel_pedals.sample(1)['frame'].values[0]

img = cv2.imread(f'../data/tmp_frames/frame_{sample_frame}.jpg')

# does not depend on the monitor resolution
scale = find_scale_factor(img, monitor_width, monitor_height)
img_rescaled = rescale_frame(img, scale=scale)

def capture_event(event, x, y, flags, param):
    captured = False

    if event == cv2.EVENT_MOUSEMOVE:

        # make a copy of the image
        img_rescaled_tmp = img_rescaled.copy()

        # plot a green vertical and horizontal line
        cv2.line(img_rescaled_tmp, (x, 0), (x, img_rescaled.shape[0]), (0, 255, 0), 1)
        cv2.line(img_rescaled_tmp, (0, y), (img_rescaled.shape[1], y), (0, 255, 0), 1)

        cv2.imshow("frame", img_rescaled_tmp)

    if event == cv2.EVENT_LBUTTONDOWN:
        print(f"Captured coordinates ({x}, {y}), press any key to exit")


cv2.imshow('frame', img_rescaled)
cv2.setMouseCallback('frame', capture_event)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

[ WARN:0@0.960] global loadsave.cpp:241 findDecoder imread_('../data/tmp_frames/frame_     frame                          hip                         knee  \
892    893  [229.61586714 317.24868774]  [170.38742423 358.6038208 ]   
405    406  [228.17612171 317.03281403]  [168.78598452 353.87317657]   
844    845  [228.27789545 319.14611816]  [167.3450911  352.20867157]   

                           ankle                  ankle_right  \
892  [183.11395884 431.86344147]  [222.36270189 431.12194061]   
405  [186.63378954 430.67718506]  [221.4908123  431.07147217]   
844  [183.46150875 430.84754944]  [217.89315462 430.06950378]   

                        shoulder                        elbow  \
892  [170.76305151 272.1087265 ]  [148.67548943 318.97632599]   
405  [172.47549176 260.67001343]  [143.31851721 306.77845001]   
844  [163.563627   281.40497208]  [151.84125781 331.55124664]   

                           wrist  angle_knee  angle_shoulder  angle_elbow  \
892  [103.3280468  334.589

AttributeError: 'NoneType' object has no attribute 'shape'